# `pymathutils.mesh` submodule tests

In [ ]:
import sys
from pathlib import Path

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))

# ply_dir = (proj_dir / "data" / "example_ply").resolve()
ply_dir = (proj_dir / "data" / "example_ply_jan_15_26").resolve()
out_dir = (proj_dir / "output").resolve()

hex_patch_he_ply = f"{ply_dir}/hex_patch_he.ply"
hex_patch_vf_ply = f"{ply_dir}/hex_patch_vf.ply"
annulus_he_ply = f"{ply_dir}/annulus_he.ply"
annulus_vf_ply = f"{ply_dir}/annulus_vf.ply"
plane_6_he_ply = f"{ply_dir}/nice_plane/plane_0000006_he.ply"
plane_24_ply = f"{ply_dir}/nice_plane/plane_0000024_he.ply"
dumbbell_coarse_he_ply = f"{ply_dir}/dumbbell_coarse_he.ply"


## Input/Output

### `C++` $\rightarrow$ `Python` bindings

- `load_mesh_samples_from_ply`
- `write_mesh_samples_to_ply`
- `load_mesh_samples`
- `save_mesh_samples`

#### `write_mesh_samples_to_ply` zero-length array bug

Works unless: array length is zero and dtype is int

In [ ]:
import sys
from pathlib import Path
import os

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))
in_ply_dir = (proj_dir / "data" / "example_ply").resolve()
out_ply_dir = (proj_dir / "output" / "example_ply").resolve()
os.system(f"mkdir -p {out_ply_dir}")

from pymathutils.mesh import load_mesh_samples_from_ply, write_mesh_samples_to_ply
import numpy as np

inply = f"{in_ply_dir}/nice_plane/plane_0000006_he.ply"
outply = f"{out_ply_dir}/plane_0000006_he.ply"

s0 = load_mesh_samples_from_ply(inply)
s0["h_negative_B"] = np.zeros((1,), dtype=int) # nonzero length, dtype=int
write_mesh_samples_to_ply(s0, outply)
s0 = load_mesh_samples_from_ply(inply)
s0["h_negative_B"] = np.zeros((0,), dtype=np.intc) # zero length, dtype=np.intc
write_mesh_samples_to_ply(s0, outply)
s0 = load_mesh_samples_from_ply(inply)
s0["h_negative_B"] = np.zeros((0,), dtype=np.int32) # zero length, dtype=np.int32
write_mesh_samples_to_ply(s0, outply)
print("those were fine")

print("this is not")
s0 = load_mesh_samples_from_ply(inply)
s0["h_negative_B"] = np.zeros((0,), dtype=int) # zero length, dtype=int
write_mesh_samples_to_ply(s0, outply)
print("actually, just kidding. that was fine too")

#### Combined save/load tests `load_mesh_samples_from_ply`, `write_mesh_samples_to_ply`

In [ ]:
import sys
from pathlib import Path
import os

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))
in_ply_dir = (proj_dir / "data" / "example_ply").resolve()
out_ply_dir = (proj_dir / "output" / "example_ply").resolve()
os.system(f"mkdir -p {out_ply_dir}")

from pymathutils.mesh import load_mesh_samples_from_ply, write_mesh_samples_to_ply
import numpy as np

inply = f"{in_ply_dir}/dumbbell_coarse_he.ply"
outply = f"{out_ply_dir}/dumbbell_coarse_he.ply"

# inply = f"{proj_dir}/data/example_ply/annulus_he.ply"
# outply = f"{proj_dir}/output/annulus_he.ply"

print("\nTest raw ply")
s0 = load_mesh_samples_from_ply(inply)
print(f"{s0.keys()=}")
write_mesh_samples_to_ply(s0, outply)
s1 = load_mesh_samples_from_ply(outply)
print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
shapes0 = [s0[k].shape for k in s0.keys()]
shapes1 = [s0[k].shape for k in s0.keys()]
assert np.all([a==b for a,b in zip(shapes0, shapes1)])
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0

print("\nTest saved ply")
s0 = s1
print(f"{s0.keys()=}")
write_mesh_samples_to_ply(s0, outply)
s1 = load_mesh_samples_from_ply(outply)
print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0


#### Combined save/load tests `load_mesh_samples`, `save_mesh_samples`

Still crashes sometimes? Hard to reproduce consistently. Seems kinda random.

In [ ]:
import sys
from pathlib import Path
import os

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))
in_ply_dir = (proj_dir / "data" / "example_ply_feb_16_26").resolve()
out_ply_dir = (proj_dir / "output" / "example_ply").resolve()
os.system(f"mkdir -p {out_ply_dir}")

from pymathutils.mesh import load_mesh_samples, save_mesh_samples
import numpy as np

inply = f"{in_ply_dir}/nice_plane/plane_0000006_he.ply"
outply = f"{out_ply_dir}/plane_0000006_he.ply"

inply = f"{in_ply_dir}/dumbbell_coarse_he.ply"
outply = f"{out_ply_dir}/dumbbell_coarse_he.ply"

# inply = f"{in_ply_dir}/annulus_he.ply"
# outply = f"{out_ply_dir}/annulus_he.ply"

print("\nTest raw ply")
s0 = load_mesh_samples(inply)
print(f"{s0.keys()=}")
save_mesh_samples(s0, outply)
s1 = load_mesh_samples(outply)
print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
shapes0 = [s0[k].shape for k in s0.keys()]
shapes1 = [s0[k].shape for k in s0.keys()]
assert np.all([a==b for a,b in zip(shapes0, shapes1)])
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0

print("\nTest saved ply")
s0 = s1
print(f"{s0.keys()=}")
save_mesh_samples(s0, outply)
s1 = load_mesh_samples(outply)
print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
shapes0 = [s0[k].shape for k in s0.keys()]
shapes1 = [s0[k].shape for k in s0.keys()]
assert np.all([a==b for a,b in zip(shapes0, shapes1)])
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0


### `pymathutils.mesh.pyutils` Python implementations

- `HalfEdgeMesh`
  - `load_ply`
  - `save_ply`

In [ ]:
import sys
from pathlib import Path

nb_dir = Path().resolve()
proj_dir = (nb_dir / "..").resolve()
sys.path.insert(0, str(proj_dir))

# ply_dir = (proj_dir / "data" / "example_ply").resolve()
ply_dir = (proj_dir / "data" / "example_ply_jan_15_26").resolve()
out_dir = (proj_dir / "output").resolve()

from pymathutils.mesh.pyutils import HalfEdgeMesh
import numpy as np
from pymathutils.mesh import load_mesh_samples_from_ply, write_mesh_samples_to_ply

# inply = f"{ply_dir}/hex_patch_he.ply"
inply = f"{ply_dir}/dumbbell_coarse_he.ply"

outply = f"{proj_dir}/output/dumbbell_coarse_he.ply"


print("\nTest raw ply")
m0 = HalfEdgeMesh.load_ply(inply)
s0 = m0.mesh_samples
print(f"{s0.keys()=}")
for k,v in s0.items():
    print(f"{k}: {v.dtype=}")
s0["h_negative_B"] = np.array([0], dtype=int)
write_mesh_samples_to_ply(s0, outply)
m0.save_ply(outply)

In [ ]:
from pymathutils.mesh.pyutils import HalfEdgeMesh
import numpy as np

inply = dumbbell_coarse_he_ply
outply = f"{proj_dir}/output/dumbbell_coarse_he.ply"


print("\nTest raw ply")
m0 = HalfEdgeMesh.load_ply(inply)
s0 = m0.mesh_samples
print(f"{s0.keys()=}")
m0.save_ply(outply)
m1 = HalfEdgeMesh.load_ply(outply)
s1 = m1.mesh_samples

print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0

print("\nTest saved ply")
m0 = m1
s0 = m0.mesh_samples
print(f"{s0.keys()=}")
m1 = HalfEdgeMesh.load_ply(outply)
s1 = m1.mesh_samples

print(f"{s1.keys()=}")
assert s0.keys() == s1.keys()
diffs0 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s0.keys()])
diffs1 = np.linalg.norm([np.linalg.norm(s1[k]-s0[k]) for k in s1.keys()])
print(f"{diffs0=}")
print(f"{diffs1=}")
assert diffs0 == 0
assert diffs1 == 0

## Visualization

### `pymathutils.mesh` core implementations

### `pymathutils.mesh.pyutils` Python implementations

In [ ]:

def plot_hem0(m):
    pts_V, vecs_V = get_half_edge_vector_field(m)

    pv_m = pv.PolyData(m.xyz_coord_V, faces=np.hstack([np.full((len(m.V_cycle_F), 1), 3), m.V_cycle_F]))
    
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(pv_m, color="lightblue", show_edges=True)
    plotter.add_arrows(pts_V, vecs_V, mag=1.0, color="red")
    plotter.show(jupyter_backend='trame')
    return plotter

def plot_hem(m):
    import pyvista as pv
    from mesh_viewer import get_half_edge_vector_field
    import numpy as np
    # Define PyVista surface mesh
    pv_m = pv.PolyData(m.xyz_coord_V, faces=np.hstack([np.full((len(m.V_cycle_F), 1), 3), m.V_cycle_F]))
    # Define PyVista half-edge vector field
    pts_H, vecs_H = get_half_edge_vector_field(m)
    vecs_H_source = pv.Arrow(
    tip_length=0.125,      # Arrow tip length (0.0-1.0)
    tip_radius=0.025,       # Arrow tip radius
    tip_resolution=5,    # Tip smoothness
    shaft_radius=0.00625,   # Shaft thickness
    shaft_resolution=3,   # Shaft smoothness
    )
    pv_pts_H = pv.PolyData(pts_H)
    pv_pts_H['half_edges'] = vecs_H
    pv_H = pv_pts_H.glyph(orient='half_edges', scale='half_edges', factor=1, geom=vecs_H_source,)
    
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(pv_m, color="lightblue", show_edges=True)
    plotter.add_mesh(pv_H, color="red")
    plotter.show(jupyter_backend='trame')
    return plotter